In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks

# Load Data
train_df = pd.read_csv("modified_data_train.csv")
test_df = pd.read_csv("modified_data_test.csv")

print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)

# Split features and target
TARGET_COL = 'faultNumber'  
X_train = train_df.drop(columns=[TARGET_COL])
y_train = train_df[TARGET_COL]

X_test = test_df.drop(columns=[TARGET_COL])
y_test = test_df[TARGET_COL]


Train shape: (25920, 41)
Test shape: (50760, 41)


In [2]:
# Encode target if categorical
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)
num_classes = len(np.unique(y_train))
print(f"\nDetected {num_classes} class(es).")


Detected 18 class(es).


In [3]:
# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [4]:
# Build ANN model
def build_ann(input_dim, num_classes):
    model = models.Sequential()
    model.add(layers.Input(shape=(input_dim,)))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.2))
    
    # Output layer based on classification type
    if num_classes == 2:
        model.add(layers.Dense(1, activation='sigmoid'))  # Binary
        loss = 'binary_crossentropy'
        metrics = ['accuracy']
    else:
        model.add(layers.Dense(num_classes, activation='softmax'))  # Multi-class
        loss = 'sparse_categorical_crossentropy'
        metrics = ['accuracy']
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss=loss,
        metrics=metrics
    )
    return model

model = build_ann(X_train_scaled.shape[1], num_classes)
model.summary()

# Callbacks
early_stop = callbacks.EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, min_lr=1e-6)

# Train Model
history = model.fit(
    X_train_scaled, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=32,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         5,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 18)             │         1,170 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,674 (57.32 KB)

 Trainable params: 14,674 (57.32 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
648/648 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3516 - loss: 2.1033 - val_accuracy: 0.3119 - val_loss: 2.1185 - learning_rate: 0.0010
Epoch 2/50
648/648 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5867 - loss: 1.2181 - val_accuracy: 0.3962 - val_loss: 2.0404 - learning_rate: 0.0010
Epoch 3/50
648/648 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6749 - loss: 0.9610 - val_accuracy: 0.4286 - val_loss: 2.0745 - learning_rate: 0.0010
Epoch 4/50
648/648 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7066 - loss: 0.8308 - val_accuracy: 0.4217 - val_loss: 2.4332 - learning_rate: 0.0010
Epoch 5/50
648/648 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7371 - loss: 0.7390 - val_accuracy: 0.4223 - val_loss: 2.6827 - learning_rate: 0.0010
Epoch 6/50
648/648 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7441 - loss: 0.7020 - val_accuracy: 0.4479 - val_loss: 2.6853 - learning_rate: 0.0010
Epoch 7/50
648/648 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7595 - loss: 0.6550 - 

In [5]:
# Evaluate Model
if num_classes == 2:
    y_pred_prob = model.predict(X_test_scaled)
    y_pred = (y_pred_prob > 0.5).astype(int).flatten()
else:
    y_pred_prob = model.predict(X_test_scaled)
    y_pred = np.argmax(y_pred_prob, axis=1)

# Metrics
print("\n--- Evaluation Results ---")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=le.classes_.astype(str)))

# Save Model and Scaler
model.save("fault_detection_ann_model.keras")

import joblib
joblib.dump(scaler, "fault_scaler.joblib")
joblib.dump(le, "fault_label_encoder.joblib")

print("\n Model, Scaler, and Label Encoder saved successfully!")

1587/1587 ━━━━━━━━━━━━━━━━━━━━ 1s 691us/step

--- Evaluation Results ---
Accuracy: 0.5043932230102443

Confusion Matrix:
 [[1744    0    2  113  180    0    0    0  231   62    0    1    0    8
     1    0  254  224]
 [ 195 2366    2   49   59    0    0    0   68   20    1    0    0    0
     1    0   33   26]
 [ 202    0 2371   50   51    0    0    4   74   20    0    0    0    0
     1    0   33   14]
 [ 207    0    2 2378   49    0    0    0   71   23    0    0    3    0
     1    0   72   14]
 [1340    0    2   85  517    0    1   39  238   51  115   11    0    8
     1    0  222  190]
 [ 187    0    2   49   47 1736  156  158   68   20  130  220    0    0
     1    0   33   13]
 [ 187    0    2   49   47    0 2400    0   68   20    0    0    0    0
     1    0   33   13]
 [ 217  147   25   67  128    0    6 1416  116   24  507   43    0    4
     1   15   39   65]
 [1381    0    2  109  444    0    0    0  331   55    0    2    0   13
     1    0  239  243]
 [ 758    0    2  513  